# Negative Prompting and Avoiding Undesired Outputs

As AI language models become more powerful, it's crucial to guide their outputs effectively. Negative prompting allows us to specify what we don't want in the model's responses, helping to refine and control the generated content. This approach is particularly useful when dealing with sensitive topics, ensuring factual accuracy, or maintaining a specific tone or style in the output.

## Key Components
1. Using negative examples to guide the model
2. Specifying exclusions in prompts
3. Implementing constraints using LangChain
4. Evaluating and refining negative prompts

In [1]:
! pip install langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 932.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

os.environ['GOOGLE_API_KEY']=''

# Inatiate the LLM
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

def get_response(prompt):
  """ Helper function to get model completion."""
  return llm.invoke(prompt).content

# 1. Using Negative Examples

In [4]:
negative_example_prompt=PromptTemplate(
    input_variables=['topic'],
    template="""
        Provide a brief explanation of {topic}.
        Do Not include any of following in your explanation:
        - Technical jargon or complex terminology
        - Historical background or dates
        - Comparisons to other related topics
        Your explanation should be simple,direct and focus only on the core concept.
    """
)
response=get_response(negative_example_prompt.format(topic='Photosynthesis'))
print(response)

Plants use sunlight, water, and air to create their own food.  This food gives them energy to grow.


# 2. Specifying Exclusions

In [5]:
exclusion_prompt=PromptTemplate(
    input_variables=['topic','exclude'],
    template ="""
        Write a short paragraph about {topic}.
        Important: DO not mention or reference anything related to {exclude}.

    """
)

response=get_response(exclusion_prompt.format(topic='The benefits of exercise',exclude='Weight loss or body image'))
print(response)


Regular exercise offers a multitude of benefits beyond physical appearance.  It significantly improves cardiovascular health, strengthening the heart and lungs and reducing the risk of chronic diseases.  Exercise boosts mood and reduces stress and anxiety by releasing endorphins.  Furthermore, it enhances sleep quality, improves cognitive function, and strengthens bones and muscles, contributing to overall physical strength and resilience.  A consistent exercise routine fosters a sense of well-being and increases energy levels throughout the day.


# Implementing Constraints

In [6]:
constraint_prompt = PromptTemplate(
    input_variables=["topic", "style", "excluded_words"],
    template="""Write a {style} description of {topic}.
    Constraints:
    1. Do not use any of these words: {excluded_words}
    2. Keep the description under 100 words
    3. Do not use analogies or metaphors
    4. Focus only on factual information"""
)

response = get_response(constraint_prompt.format(
    topic="artificial intelligence",
    style="technical",
    excluded_words="robot, human-like, science fiction"
))
print(response)

Artificial intelligence (AI) encompasses computational systems capable of performing tasks typically requiring human intelligence.  These tasks include learning, reasoning, problem-solving, perception, and natural language processing. AI systems achieve this through algorithms and statistical models trained on large datasets.  Their performance is measured by accuracy and efficiency in completing specified objectives.  Subfields include machine learning, deep learning, and computer vision.


# 4. Evaluation and Refinment

In [8]:
import re

def evaluate_output(output, constraints):
    """Evaluate if the output meets the given constraints."""
    results = {}
    for constraint, check_func in constraints.items():
        results[constraint] = check_func(output)
    return results

# Define some example constraints
constraints = {
    "word_count": lambda x: len(x.split()) <= 100,
    "no_excluded_words": lambda x: all(word not in x.lower() for word in ["robot", "human-like", "science fiction"]),
    "no_analogies": lambda x: not re.search(r"\b(as|like)\b", x, re.IGNORECASE)

}

# Evaluate the previous output
evaluation_results = evaluate_output(response, constraints)
print("Evaluation results:", evaluation_results)

# If the output doesn't meet all constraints, we can refine our prompt
if not all(evaluation_results.values()):
    refined_prompt = constraint_prompt.format(
        topic="artificial intelligence",
        style="technical and concise",  # Added 'concise' to address word count
        excluded_words="robot, human-like, science fiction, like, as"  # Added 'like' and 'as' to avoid analogies
    )
    refined_response = get_response(refined_prompt)
    print("\nRefined response:\n", refined_response)

    # Evaluate the refined output
    refined_evaluation = evaluate_output(refined_response, constraints)
    print("\nRefined evaluation results:", refined_evaluation)

Evaluation results: {'word_count': True, 'no_excluded_words': True, 'no_analogies': True}
